#### Importing the necessary libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import tensorflow as tf
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tensorflow import keras
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Sequential
from keras import layers
import string

[nltk_data] Downloading package stopwords to /home/kvp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


#### Reading the data

In [4]:
import os,glob
folder_path ='MT'
i=0
MT_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        MT_sents.extend(f.read().split("\n"))
MT_sents=list(set(MT_sents))

folder_path ='OC'
i=0
OC_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        OC_sents.extend(f.read().split("\n"))
OC_sents=list(set(OC_sents))

folder_path ='PE'
i=0
PE_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        PE_sents.extend(f.read().split("\n"))
PE_sents=list(set(PE_sents))
        
folder_path ='VG'
i=0
VG_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        VG_sents.extend(f.read().split("\n"))
VG_sents=list(set(VG_sents))
        
folder_path ='WD'
i=0
WD_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        WD_sents.extend(f.read().split("\n"))
WD_sents=list(set(WD_sents))

folder_path ='WTP'
i=0
WTP_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        WTP_sents.extend(f.read().split("\n"))
WTP_sents=list(set(WTP_sents))

In [5]:
print(len(MT_sents))
print(len(OC_sents))
print(len(PE_sents))
print(len(VG_sents))
print(len(WD_sents))
print(len(WTP_sents))

450
8529
7052
2577
3873
8411


#### Data Preprocessing and Cleaning

In [6]:
MT_s=[]
MT_l=[]
for i in MT_sents:
    if len(i)>1:
        MT_s.append(i.split("\t")[0])
        MT_l.append(int(i.split("\t")[1]))

        
OC_s=[]
OC_l=[]
for i in OC_sents:
    if len(i)>1:
        OC_s.append(i.split("\t")[0])
        OC_l.append(int(i.split("\t")[1]))

        
PE_s=[]
PE_l=[]
for i in PE_sents:
    if len(i)>1:
        PE_s.append(i.split("\t")[0])
        PE_l.append(int(i.split("\t")[1]))
        

VG_s=[]
VG_l=[]
for i in VG_sents:
    if len(i)>1:
        VG_s.append(i.split("\t")[0])
        VG_l.append(int(i.split("\t")[1]))
        
        
WD_s=[]
WD_l=[]
for i in WD_sents:
    if len(i)>1:
        WD_s.append(i.split("\t")[0])
        WD_l.append(int(i.split("\t")[1]))
        
        
WTP_s=[]
WTP_l=[]
for i in WTP_sents:
    if len(i)>1:
        WTP_s.append(i.split("\t")[0])
        WTP_l.append(int(i.split("\t")[1]))

In [7]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text


#### Converting the Data into a pandas dataframes

In [8]:
df_MT=pd.DataFrame({'sentence':MT_s,'label':MT_l})
df_OC=pd.DataFrame({'sentence':OC_s,'label':OC_l})
df_PE=pd.DataFrame({'sentence':PE_s,'label':PE_l})
df_VG=pd.DataFrame({'sentence':VG_s,'label':VG_l})
df_WD=pd.DataFrame({'sentence':WD_s,'label':WD_l})
df_WTP=pd.DataFrame({'sentence':WTP_s,'label':WTP_l})

In [19]:
df_MT['sentence'] = df_MT['sentence'].map(lambda x: clean_text(x))
df_OC['sentence'] = df_OC['sentence'].map(lambda x: clean_text(x))
df_PE['sentence'] = df_PE['sentence'].map(lambda x: clean_text(x))
df_VG['sentence'] = df_VG['sentence'].map(lambda x: clean_text(x))
df_WD['sentence'] = df_WD['sentence'].map(lambda x: clean_text(x))
df_WTP['sentence'] = df_WTP['sentence'].map(lambda x: clean_text(x))

#### Showing the data

In [32]:
df_MT.head()

,sentence,label
0,good wonder outfit univ actual allow major cle...,0
1,pharmacist argu categor favour counter avail m...,1
2,fee result longer durat studi,0
3,berlin basketb team come sober season,1
4,bad howev rent suddenli climb 100 200,0


In [23]:
df_OC.head()

,sentence,label
0,offici sell get money,0
1,may well add guy girlfriend even girl girlfrie...,0
2,diz dehuman plan take advantag,0
3,sure made avail peopl want abl buy watch somet...,0
4,vote republican peopl thought well,0


In [26]:
df_PE.head()

,sentence,label
0,accordingli sale promot easi consum compar pri...,0
1,thu plan renew resourc energi may face challen...,0
2,whether person social achiev man laudabl,0
3,first stori end way bore,0
4,look anoth perspect written word also vital or...,1


In [27]:
df_VG.head()

,sentence,label
0,grennan inform logic mcgill queen univ press m...,0
1,wick action judg result wick impli,1
2,think govern take control land way land distri...,1
3,reali intern one deci quit drug cannot enforc ...,0
4,junctur need lot money reach peopl put forth i...,1


In [28]:
df_WD.head()

,sentence,label
0,public school parent teacher much say govern r...,0
1,spontan erupt religio ffrf find threaten stude...,0
2,offici school question event share letter,0
3,wish move sooner could afford privat school so...,0
4,better bond understand child need close avail ...,0


In [29]:
df_WTP.head()

,sentence,label
0,heard gap stand gay apparel someth,0
1,crazi talk,0
2,discuss topic particip,0
3,northern ireland flag npov issu,0
4,put number analysi,0


In [25]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#### In Domain Analysis

#### For each dataset, we will be training 20 models and in each model we will discard certain negative samples inorder to tackle the data imbalance and have no loss of information

#### On MT dataset

In [11]:
f1_MT=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_MT[df_MT["label"]==1]
    df_negative=df_MT[df_MT["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x=df.sentence.values
    y=df.label.values

    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=123)

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_MT.append(result["macro avg"]["f1-score"])

Model 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            48150     
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 70,8

Epoch 1/20
201/201 [==============================] - 1s 4ms/step - loss: 0.6943 - accuracy: 0.4577
Epoch 2/20
201/201 [==============================] - 0s 716us/step - loss: 0.6974 - accuracy: 0.4527
Epoch 3/20
201/201 [==============================] - 0s 724us/step - loss: 0.6883 - accuracy: 0.5373
Epoch 4/20
201/201 [==============================] - 0s 779us/step - loss: 0.6847 - accuracy: 0.5821
Epoch 5/20
201/201 [==============================] - 0s 1ms/step - loss: 0.6932 - accuracy: 0.5323
Epoch 6/20
201/201 [==============================] - 0s 943us/step - loss: 0.6740 - accuracy: 0.5721
Epoch 7/20
201/201 [==============================] - 0s 867us/step - loss: 0.6065 - accuracy: 0.7761
Epoch 8/20
201/201 [==============================] - 0s 891us/step - loss: 0.4821 - accuracy: 0.7960
Epoch 9/20
201/201 [==============================] - 0s 754us/step - loss: 0.4769 - accuracy: 0.8159
Epoch 10/20
201/201 [==============================] - 0s 1ms/step - loss: 0.4093 - ac

201/201 [==============================] - 0s 745us/step - loss: 0.3005 - accuracy: 0.9055
Epoch 13/20
201/201 [==============================] - 0s 710us/step - loss: 0.3131 - accuracy: 0.9005
Epoch 14/20
201/201 [==============================] - 0s 708us/step - loss: 0.2762 - accuracy: 0.9254
Epoch 15/20
201/201 [==============================] - 0s 704us/step - loss: 0.2948 - accuracy: 0.9104
Epoch 16/20
201/201 [==============================] - 0s 704us/step - loss: 0.2859 - accuracy: 0.9055
Epoch 17/20
201/201 [==============================] - 0s 694us/step - loss: 0.3074 - accuracy: 0.9055
Epoch 18/20
201/201 [==============================] - 0s 707us/step - loss: 0.2967 - accuracy: 0.9055
Epoch 19/20
201/201 [==============================] - 0s 699us/step - loss: 0.2581 - accuracy: 0.9204
Epoch 20/20
201/201 [==============================] - 0s 728us/step - loss: 0.2455 - accuracy: 0.9204
              precision    recall  f1-score   support

           0       0.91      0

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 50)            49150     
_________________________________________________________________
lstm_15 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_16 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_8 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 9         
Total params: 71,887
Trainable params: 71,887
Non-trainable params: 0
__________________________________________________

Epoch 1/20
201/201 [==============================] - 1s 4ms/step - loss: 0.6951 - accuracy: 0.4975
Epoch 2/20
201/201 [==============================] - 0s 724us/step - loss: 0.6931 - accuracy: 0.5224
Epoch 3/20
201/201 [==============================] - 0s 729us/step - loss: 0.6973 - accuracy: 0.4677
Epoch 4/20
201/201 [==============================] - 0s 713us/step - loss: 0.6914 - accuracy: 0.5373
Epoch 5/20
201/201 [==============================] - 0s 787us/step - loss: 0.6896 - accuracy: 0.5373
Epoch 6/20
201/201 [==============================] - 0s 903us/step - loss: 0.6442 - accuracy: 0.6119
Epoch 7/20
201/201 [==============================] - 0s 740us/step - loss: 0.5203 - accuracy: 0.7612
Epoch 8/20
201/201 [==============================] - 0s 752us/step - loss: 0.5359 - accuracy: 0.7662
Epoch 9/20
201/201 [==============================] - 0s 789us/step - loss: 0.5355 - accuracy: 0.7861
Epoch 10/20
201/201 [==============================] - 0s 753us/step - loss: 0.4542 

201/201 [==============================] - 0s 787us/step - loss: 0.5417 - accuracy: 0.7413
Epoch 13/20
201/201 [==============================] - 0s 769us/step - loss: 0.4905 - accuracy: 0.7811
Epoch 14/20
201/201 [==============================] - 0s 789us/step - loss: 0.4783 - accuracy: 0.7562
Epoch 15/20
201/201 [==============================] - 0s 776us/step - loss: 0.4721 - accuracy: 0.7910
Epoch 16/20
201/201 [==============================] - 0s 806us/step - loss: 0.4430 - accuracy: 0.8060
Epoch 17/20
201/201 [==============================] - 0s 796us/step - loss: 0.4696 - accuracy: 0.7910
Epoch 18/20
201/201 [==============================] - 0s 793us/step - loss: 0.4054 - accuracy: 0.8209
Epoch 19/20
201/201 [==============================] - 0s 788us/step - loss: 0.4122 - accuracy: 0.8209
Epoch 20/20
201/201 [==============================] - 0s 801us/step - loss: 0.4198 - accuracy: 0.8209
              precision    recall  f1-score   support

           0       0.73      0

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 20, 50)            49300     
_________________________________________________________________
lstm_29 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_30 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_15 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 9         
Total params: 72,037
Trainable params: 72,037
Non-trainable params: 0
_________________________________________________

Epoch 1/20
201/201 [==============================] - 1s 5ms/step - loss: 0.6963 - accuracy: 0.4627
Epoch 2/20
201/201 [==============================] - 0s 751us/step - loss: 0.6953 - accuracy: 0.5075
Epoch 3/20
201/201 [==============================] - 0s 755us/step - loss: 0.6925 - accuracy: 0.5373
Epoch 4/20
201/201 [==============================] - 0s 768us/step - loss: 0.6944 - accuracy: 0.4925
Epoch 5/20
201/201 [==============================] - 0s 757us/step - loss: 0.6959 - accuracy: 0.4677
Epoch 6/20
201/201 [==============================] - 0s 798us/step - loss: 0.6920 - accuracy: 0.5224
Epoch 7/20
201/201 [==============================] - 0s 793us/step - loss: 0.6911 - accuracy: 0.5224
Epoch 8/20
201/201 [==============================] - 0s 799us/step - loss: 0.6928 - accuracy: 0.5373
Epoch 9/20
201/201 [==============================] - 0s 767us/step - loss: 0.6906 - accuracy: 0.5124
Epoch 10/20
201/201 [==============================] - 0s 805us/step - loss: 0.6903 

201/201 [==============================] - 0s 772us/step - loss: 0.3810 - accuracy: 0.8706
Epoch 13/20
201/201 [==============================] - 0s 998us/step - loss: 0.3806 - accuracy: 0.8756
Epoch 14/20
201/201 [==============================] - 0s 1ms/step - loss: 0.2918 - accuracy: 0.9104
Epoch 15/20
201/201 [==============================] - 0s 933us/step - loss: 0.2860 - accuracy: 0.9104
Epoch 16/20
201/201 [==============================] - 0s 1ms/step - loss: 0.3161 - accuracy: 0.8955
Epoch 17/20
201/201 [==============================] - 0s 893us/step - loss: 0.3050 - accuracy: 0.9154
Epoch 18/20
201/201 [==============================] - 0s 945us/step - loss: 0.3312 - accuracy: 0.9005
Epoch 19/20
201/201 [==============================] - 0s 1ms/step - loss: 0.3251 - accuracy: 0.8955
Epoch 20/20
201/201 [==============================] - 0s 896us/step - loss: 0.2777 - accuracy: 0.9303
              precision    recall  f1-score   support

           0       0.92      0.86   

In [12]:
print(f"Macro avg f1 score on the MT dataset is {sum(f1_MT)/len(f1_MT)}")

Macro avg f1 score on the MT dataset is 0.7055349174561594


#### On OC dataset

In [13]:
f1_OC=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_OC[df_OC["label"]==1]
    df_negative=df_OC[df_OC["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x=df.sentence.values
    y=df.label.values

    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=123)

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_OC.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 20, 50)            162900    
_________________________________________________________________
lstm_41 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_42 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_21 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 9         
Total params: 185,637
Trainable params: 185,637
Non-trainable params: 0
_______________________________________

Epoch 1/20
1263/1263 [==============================] - 2s 2ms/step - loss: 0.6956 - accuracy: 0.5012
Epoch 2/20
1263/1263 [==============================] - 1s 881us/step - loss: 0.6952 - accuracy: 0.4941
Epoch 3/20
1263/1263 [==============================] - 1s 916us/step - loss: 0.6853 - accuracy: 0.5495
Epoch 4/20
1263/1263 [==============================] - 1s 921us/step - loss: 0.6649 - accuracy: 0.6207
Epoch 5/20
1263/1263 [==============================] - 1s 904us/step - loss: 0.6600 - accuracy: 0.6247
Epoch 6/20
1263/1263 [==============================] - 1s 949us/step - loss: 0.6571 - accuracy: 0.6397
Epoch 7/20
1263/1263 [==============================] - 1s 906us/step - loss: 0.6455 - accuracy: 0.6477
Epoch 8/20
1263/1263 [==============================] - 1s 851us/step - loss: 0.6459 - accuracy: 0.6572
Epoch 9/20
1263/1263 [==============================] - 1s 891us/step - loss: 0.6398 - accuracy: 0.6524
Epoch 10/20
1263/1263 [==============================] - 1s 888us/

1263/1263 [==============================] - 1s 861us/step - loss: 0.6335 - accuracy: 0.6714
Epoch 12/20
1263/1263 [==============================] - 1s 899us/step - loss: 0.6309 - accuracy: 0.6746
Epoch 13/20
1263/1263 [==============================] - 1s 940us/step - loss: 0.6245 - accuracy: 0.6809
Epoch 14/20
1263/1263 [==============================] - 1s 935us/step - loss: 0.6299 - accuracy: 0.6714
Epoch 15/20
1263/1263 [==============================] - 1s 942us/step - loss: 0.6163 - accuracy: 0.6817
Epoch 16/20
1263/1263 [==============================] - 1s 928us/step - loss: 0.6182 - accuracy: 0.6833
Epoch 17/20
1263/1263 [==============================] - 1s 911us/step - loss: 0.6224 - accuracy: 0.6841
Epoch 18/20
1263/1263 [==============================] - 1s 906us/step - loss: 0.6155 - accuracy: 0.6896
Epoch 19/20
1263/1263 [==============================] - 1s 906us/step - loss: 0.6115 - accuracy: 0.6880
Epoch 20/20
1263/1263 [==============================] - 1s 940us/s

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 20, 50)            160500    
_________________________________________________________________
lstm_55 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_56 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_28 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_56 (Dense)             (None, 1)                 9         
Total params: 183,237
Trainable params: 183,237
Non-trainable params: 0
_______________________________________________

Epoch 1/20
1263/1263 [==============================] - 2s 2ms/step - loss: 0.6971 - accuracy: 0.4925
Epoch 2/20
1263/1263 [==============================] - 1s 878us/step - loss: 0.6937 - accuracy: 0.4861
Epoch 3/20
1263/1263 [==============================] - 1s 914us/step - loss: 0.6930 - accuracy: 0.5273
Epoch 4/20
1263/1263 [==============================] - 1s 962us/step - loss: 0.6751 - accuracy: 0.6010
Epoch 5/20
1263/1263 [==============================] - 1s 962us/step - loss: 0.6509 - accuracy: 0.6382
Epoch 6/20
1263/1263 [==============================] - 1s 923us/step - loss: 0.6378 - accuracy: 0.6516
Epoch 7/20
1263/1263 [==============================] - 1s 943us/step - loss: 0.6425 - accuracy: 0.6564
Epoch 8/20
1263/1263 [==============================] - 1s 959us/step - loss: 0.6283 - accuracy: 0.6809
Epoch 9/20
1263/1263 [==============================] - 1s 967us/step - loss: 0.6226 - accuracy: 0.6888
Epoch 10/20
1263/1263 [==============================] - 1s 955us/

1263/1263 [==============================] - 1s 862us/step - loss: 0.6312 - accuracy: 0.6683
Epoch 12/20
1263/1263 [==============================] - 1s 837us/step - loss: 0.6259 - accuracy: 0.6643
Epoch 13/20
1263/1263 [==============================] - 1s 855us/step - loss: 0.6249 - accuracy: 0.6738
Epoch 14/20
1263/1263 [==============================] - 1s 842us/step - loss: 0.6203 - accuracy: 0.6809
Epoch 15/20
1263/1263 [==============================] - 1s 862us/step - loss: 0.6269 - accuracy: 0.6730
Epoch 16/20
1263/1263 [==============================] - 1s 871us/step - loss: 0.6316 - accuracy: 0.6722
Epoch 17/20
1263/1263 [==============================] - 1s 865us/step - loss: 0.6259 - accuracy: 0.6833
Epoch 18/20
1263/1263 [==============================] - 1s 872us/step - loss: 0.6250 - accuracy: 0.6833
Epoch 19/20
1263/1263 [==============================] - 1s 865us/step - loss: 0.6212 - accuracy: 0.6865
Epoch 20/20
1263/1263 [==============================] - 1s 897us/s

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, 20, 50)            155600    
_________________________________________________________________
lstm_69 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_70 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_35 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_70 (Dense)             (None, 1)                 9         
Total params: 178,337
Trainable params: 178,337
Non-trainable params: 0
_______________________________________________

Epoch 1/20
1263/1263 [==============================] - 2s 2ms/step - loss: 0.6956 - accuracy: 0.4917
Epoch 2/20
1263/1263 [==============================] - 1s 923us/step - loss: 0.6939 - accuracy: 0.4956
Epoch 3/20
1263/1263 [==============================] - 1s 953us/step - loss: 0.6862 - accuracy: 0.5487
Epoch 4/20
1263/1263 [==============================] - 1s 976us/step - loss: 0.6581 - accuracy: 0.6247
Epoch 5/20
1263/1263 [==============================] - 1s 978us/step - loss: 0.6598 - accuracy: 0.6255
Epoch 6/20
1263/1263 [==============================] - 1s 974us/step - loss: 0.6451 - accuracy: 0.6413
Epoch 7/20
1263/1263 [==============================] - 1s 972us/step - loss: 0.6362 - accuracy: 0.6374
Epoch 8/20
1263/1263 [==============================] - 1s 982us/step - loss: 0.6478 - accuracy: 0.6437
Epoch 9/20
1263/1263 [==============================] - 1s 977us/step - loss: 0.6334 - accuracy: 0.6429
Epoch 10/20
1263/1263 [==============================] - 1s 982us/

1263/1263 [==============================] - 1s 926us/step - loss: 0.6310 - accuracy: 0.6730
Epoch 12/20
1263/1263 [==============================] - 1s 908us/step - loss: 0.6255 - accuracy: 0.6849
Epoch 13/20
1263/1263 [==============================] - 1s 883us/step - loss: 0.6263 - accuracy: 0.6825
Epoch 14/20
1263/1263 [==============================] - 1s 883us/step - loss: 0.6205 - accuracy: 0.6857
Epoch 15/20
1263/1263 [==============================] - 1s 893us/step - loss: 0.6139 - accuracy: 0.6880
Epoch 16/20
1263/1263 [==============================] - 1s 889us/step - loss: 0.6128 - accuracy: 0.6873
Epoch 17/20
1263/1263 [==============================] - 1s 947us/step - loss: 0.6300 - accuracy: 0.6754
Epoch 18/20
1263/1263 [==============================] - 1s 972us/step - loss: 0.6240 - accuracy: 0.6888
Epoch 19/20
1263/1263 [==============================] - 1s 968us/step - loss: 0.6169 - accuracy: 0.6888
Epoch 20/20
1263/1263 [==============================] - 1s 966us/s

In [14]:
print(f"Macro avg f1 score on the OC dataset is {sum(f1_OC)/len(f1_OC)}")

Macro avg f1 score on the OC dataset is 0.5475434988189924


#### On PE dataset

In [15]:
f1_PE=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_PE[df_PE["label"]==1]
    df_negative=df_PE[df_PE["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x=df.sentence.values
    y=df.label.values

    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=123)

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_PE.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     (None, 20, 50)            183950    
_________________________________________________________________
lstm_81 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_82 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_41 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_81 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_82 (Dense)             (None, 1)                 9         
Total params: 206,687
Trainable params: 206,687
Non-trainable params: 0
_______________________________________

Epoch 1/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6735 - accuracy: 0.5593
Epoch 2/20
3767/3767 [==============================] - 4s 999us/step - loss: 0.6231 - accuracy: 0.6591
Epoch 3/20
3767/3767 [==============================] - 4s 996us/step - loss: 0.6144 - accuracy: 0.6759
Epoch 4/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6067 - accuracy: 0.6780
Epoch 5/20
3767/3767 [==============================] - 4s 999us/step - loss: 0.6060 - accuracy: 0.6793
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6050 - accuracy: 0.6769
Epoch 7/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6030 - accuracy: 0.6759
Epoch 8/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5994 - accuracy: 0.6828
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5956 - accuracy: 0.6820
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss:

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_48 (Embedding)     (None, 20, 50)            183700    
_________________________________________________________________
lstm_95 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_96 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_48 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_95 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_96 (Dense)             (None, 1)                 9         
Total params: 206,437
Trainable params: 206,437
Non-trainable params: 0
_______________________________________________

Epoch 1/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6870 - accuracy: 0.5360
Epoch 2/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6275 - accuracy: 0.6629
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6126 - accuracy: 0.6793
Epoch 4/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6106 - accuracy: 0.6759
Epoch 5/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6016 - accuracy: 0.6828
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5990 - accuracy: 0.6841
Epoch 7/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5985 - accuracy: 0.6857
Epoch 8/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5937 - accuracy: 0.6838
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5952 - accuracy: 0.6809
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.587

Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_55 (Embedding)     (None, 20, 50)            182550    
_________________________________________________________________
lstm_109 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_110 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_55 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_109 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_110 (Dense)            (None, 1)                 9         
Total params: 205,287
Trainable params: 205,287
Non-trainable params: 0
_______________________________________________

Epoch 1/20
3767/3767 [==============================] - 6s 1ms/step - loss: 0.6659 - accuracy: 0.5832
Epoch 2/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6181 - accuracy: 0.6732
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6152 - accuracy: 0.6804
Epoch 4/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6127 - accuracy: 0.6860
Epoch 5/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6113 - accuracy: 0.6838
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6062 - accuracy: 0.6852
Epoch 7/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6045 - accuracy: 0.6865
Epoch 8/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6038 - accuracy: 0.6857
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5974 - accuracy: 0.6926
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.591

In [16]:
print(f"Macro avg f1 score on the PE dataset is {sum(f1_PE)/len(f1_PE)}")

Macro avg f1 score on the PE dataset is 0.6606825110201264


#### On VG dataset

In [17]:
f1_VG=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_PE[df_PE["label"]==1]
    df_negative=df_PE[df_PE["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x=df.sentence.values
    y=df.label.values

    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=123)

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_VG.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_61"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_61 (Embedding)     (None, 20, 50)            182400    
_________________________________________________________________
lstm_121 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_122 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_61 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_121 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_122 (Dense)            (None, 1)                 9         
Total params: 205,137
Trainable params: 205,137
Non-trainable params: 0
_______________________________________

Epoch 1/20
3767/3767 [==============================] - 6s 2ms/step - loss: 0.6780 - accuracy: 0.5620
Epoch 2/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6288 - accuracy: 0.6626
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6128 - accuracy: 0.6777
Epoch 4/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6084 - accuracy: 0.6836
Epoch 5/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6032 - accuracy: 0.6814
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5988 - accuracy: 0.6870
Epoch 7/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5950 - accuracy: 0.6833
Epoch 8/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5966 - accuracy: 0.6817
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5913 - accuracy: 0.6852
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.591

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_68 (Embedding)     (None, 20, 50)            183450    
_________________________________________________________________
lstm_135 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_136 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_68 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_135 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_136 (Dense)            (None, 1)                 9         
Total params: 206,187
Trainable params: 206,187
Non-trainable params: 0
_______________________________________________

Epoch 1/20
3767/3767 [==============================] - 7s 2ms/step - loss: 0.6729 - accuracy: 0.5705
Epoch 2/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6178 - accuracy: 0.6682
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6094 - accuracy: 0.6772
Epoch 4/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6054 - accuracy: 0.6777
Epoch 5/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6053 - accuracy: 0.6793
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6021 - accuracy: 0.6801
Epoch 7/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5987 - accuracy: 0.6849
Epoch 8/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5894 - accuracy: 0.6852
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5896 - accuracy: 0.6822
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.590

Model: "sequential_75"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_75 (Embedding)     (None, 20, 50)            183150    
_________________________________________________________________
lstm_149 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_150 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_75 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_149 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_150 (Dense)            (None, 1)                 9         
Total params: 205,887
Trainable params: 205,887
Non-trainable params: 0
_______________________________________________

Epoch 1/20
3767/3767 [==============================] - 7s 2ms/step - loss: 0.6786 - accuracy: 0.5400
Epoch 2/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6217 - accuracy: 0.6687
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6167 - accuracy: 0.6706
Epoch 4/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6095 - accuracy: 0.6783
Epoch 5/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6052 - accuracy: 0.6825
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6032 - accuracy: 0.6780
Epoch 7/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5991 - accuracy: 0.6817
Epoch 8/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5942 - accuracy: 0.6783
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5910 - accuracy: 0.6814
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.587

In [18]:
print(f"Macro avg f1 score on the VG dataset is {sum(f1_VG)/len(f1_VG)}")

Macro avg f1 score on the VG dataset is 0.659842095815724


#### On WD dataset

In [19]:
f1_WD=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_PE[df_PE["label"]==1]
    df_negative=df_PE[df_PE["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x=df.sentence.values
    y=df.label.values

    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=123)

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_WD.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_81 (Embedding)     (None, 20, 50)            179050    
_________________________________________________________________
lstm_161 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_162 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_81 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_161 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_162 (Dense)            (None, 1)                 9         
Total params: 201,787
Trainable params: 201,787
Non-trainable params: 0
_______________________________________

Epoch 1/20
3767/3767 [==============================] - 7s 2ms/step - loss: 0.6824 - accuracy: 0.5538
Epoch 2/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6250 - accuracy: 0.6631
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6131 - accuracy: 0.6759
Epoch 4/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6143 - accuracy: 0.6806
Epoch 5/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6130 - accuracy: 0.6767
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6050 - accuracy: 0.6860
Epoch 7/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5999 - accuracy: 0.6857
Epoch 8/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5958 - accuracy: 0.6833
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5905 - accuracy: 0.6846
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.589

Model: "sequential_88"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_88 (Embedding)     (None, 20, 50)            180350    
_________________________________________________________________
lstm_175 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_176 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_88 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_175 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_176 (Dense)            (None, 1)                 9         
Total params: 203,087
Trainable params: 203,087
Non-trainable params: 0
_______________________________________________

Epoch 1/20
3767/3767 [==============================] - 7s 2ms/step - loss: 0.6941 - accuracy: 0.4993
Epoch 2/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6619 - accuracy: 0.6111
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6242 - accuracy: 0.6767
Epoch 4/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6133 - accuracy: 0.6854
Epoch 5/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6073 - accuracy: 0.6923
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6044 - accuracy: 0.6870
Epoch 7/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6023 - accuracy: 0.6934
Epoch 8/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6001 - accuracy: 0.6899
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6009 - accuracy: 0.6942
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.596

Model: "sequential_95"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_95 (Embedding)     (None, 20, 50)            179550    
_________________________________________________________________
lstm_189 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_190 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_95 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_189 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_190 (Dense)            (None, 1)                 9         
Total params: 202,287
Trainable params: 202,287
Non-trainable params: 0
_______________________________________________

Epoch 1/20
3767/3767 [==============================] - 7s 2ms/step - loss: 0.6889 - accuracy: 0.5214
Epoch 2/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6326 - accuracy: 0.6568
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6147 - accuracy: 0.6796
Epoch 4/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6098 - accuracy: 0.6833
Epoch 5/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6061 - accuracy: 0.6825
Epoch 6/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6040 - accuracy: 0.6785
Epoch 7/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6027 - accuracy: 0.6852
Epoch 8/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.5994 - accuracy: 0.6857
Epoch 9/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.5962 - accuracy: 0.6894
Epoch 10/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.588

In [20]:
print(f"Macro avg f1 score on the WD dataset is {sum(f1_WD)/len(f1_WD)}")

Macro avg f1 score on the WD dataset is 0.6702256242952677


#### On WTP dataset

In [21]:
f1_WTP=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_PE[df_PE["label"]==1]
    df_negative=df_PE[df_PE["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x=df.sentence.values
    y=df.label.values

    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=123)

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_WTP.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_101 (Embedding)    (None, 20, 50)            183850    
_________________________________________________________________
lstm_201 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_202 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_101 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_201 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_202 (Dense)            (None, 1)                 9         
Total params: 206,587
Trainable params: 206,587
Non-trainable params: 0
______________________________________

Epoch 1/20
3767/3767 [==============================] - 8s 2ms/step - loss: 0.6846 - accuracy: 0.5360
Epoch 2/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6372 - accuracy: 0.6440
Epoch 3/20
3767/3767 [==============================] - 4s 1ms/step - loss: 0.6184 - accuracy: 0.6634
Epoch 4/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6144 - accuracy: 0.6724
Epoch 5/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6159 - accuracy: 0.6703
Epoch 6/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6100 - accuracy: 0.6793
Epoch 7/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6040 - accuracy: 0.6796
Epoch 8/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6036 - accuracy: 0.6759
Epoch 9/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6009 - accuracy: 0.6883
Epoch 10/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.595

Model: "sequential_108"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_108 (Embedding)    (None, 20, 50)            181900    
_________________________________________________________________
lstm_215 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_216 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_108 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_215 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_216 (Dense)            (None, 1)                 9         
Total params: 204,637
Trainable params: 204,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
3767/3767 [==============================] - 8s 2ms/step - loss: 0.6651 - accuracy: 0.5766
Epoch 2/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6224 - accuracy: 0.6682
Epoch 3/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6127 - accuracy: 0.6735
Epoch 4/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6123 - accuracy: 0.6767
Epoch 5/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6080 - accuracy: 0.6761
Epoch 6/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6006 - accuracy: 0.6812
Epoch 7/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6022 - accuracy: 0.6852
Epoch 8/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.5984 - accuracy: 0.6796
Epoch 9/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.5986 - accuracy: 0.6870
Epoch 10/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.592

Model: "sequential_115"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_115 (Embedding)    (None, 20, 50)            181900    
_________________________________________________________________
lstm_229 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_230 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_115 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_229 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_230 (Dense)            (None, 1)                 9         
Total params: 204,637
Trainable params: 204,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
3767/3767 [==============================] - 8s 2ms/step - loss: 0.6864 - accuracy: 0.5477
Epoch 2/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6300 - accuracy: 0.6639
Epoch 3/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6206 - accuracy: 0.6716
Epoch 4/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6173 - accuracy: 0.6812
Epoch 5/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6141 - accuracy: 0.6809
Epoch 6/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6110 - accuracy: 0.6844
Epoch 7/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6087 - accuracy: 0.6886
Epoch 8/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.6055 - accuracy: 0.6868
Epoch 9/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.5976 - accuracy: 0.6921
Epoch 10/20
3767/3767 [==============================] - 5s 1ms/step - loss: 0.599

In [22]:
print(f"Macro avg f1 score on the WTP dataset is {sum(f1_WTP)/len(f1_WTP)}")

Macro avg f1 score on the WTP dataset is 0.6615029639929193


#### Cross Domain Experiments
#### Leaving one dataset and testing on it

In [11]:
frames_MT=[df_OC,df_PE,df_VG,df_WD,df_WTP]
frames_OC=[df_MT,df_PE,df_VG,df_WD,df_WTP]
frames_PE=[df_OC,df_MT,df_VG,df_WD,df_WTP]
frames_VG=[df_OC,df_PE,df_MT,df_WD,df_WTP]
frames_WD=[df_OC,df_PE,df_VG,df_MT,df_WTP]
frames_WTP=[df_OC,df_PE,df_VG,df_WD,df_MT]

In [12]:
res_MT=pd.concat(frames_MT)
res_OC=pd.concat(frames_OC)
res_PE=pd.concat(frames_PE)
res_VG=pd.concat(frames_VG)
res_WD=pd.concat(frames_WD)
res_WTP=pd.concat(frames_WTP)

#### Leaving MT dataset and testing on it

In [25]:
f1_withoutMT=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=res_MT[res_MT["label"]==1]
    df_negative=res_MT[res_MT["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_MT.sentence.values
    y_test=df_MT.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_withoutMT.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_121"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_121 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_241 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_242 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_121 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_241 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_242 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________

Epoch 1/20
9376/9376 [==============================] - 16s 2ms/step - loss: 0.6719 - accuracy: 0.5804
Epoch 2/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6513 - accuracy: 0.6218
Epoch 3/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6463 - accuracy: 0.6279
Epoch 4/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6447 - accuracy: 0.6289
Epoch 5/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6423 - accuracy: 0.6288
Epoch 6/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6439 - accuracy: 0.6262
Epoch 7/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6404 - accuracy: 0.6303
Epoch 8/20
9376/9376 [==============================] - 13s 1ms/step - loss: 0.6388 - accuracy: 0.6356
Epoch 9/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6367 - accuracy: 0.6331
Epoch 10/20
9376/9376 [==============================] - 12s 1ms/step - l

Model: "sequential_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_128 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_255 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_256 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_128 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_255 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_256 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
9376/9376 [==============================] - 16s 2ms/step - loss: 0.6689 - accuracy: 0.5860
Epoch 2/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6466 - accuracy: 0.6355
Epoch 3/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6433 - accuracy: 0.6304
Epoch 4/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6380 - accuracy: 0.6344
Epoch 5/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6381 - accuracy: 0.6375
Epoch 6/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6367 - accuracy: 0.6377
Epoch 7/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6318 - accuracy: 0.6375
Epoch 8/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6308 - accuracy: 0.6428
Epoch 9/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6280 - accuracy: 0.6446
Epoch 10/20
9376/9376 [==============================] - 12s 1ms/step - l

Model: "sequential_135"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_135 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_269 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_270 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_135 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_269 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_270 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
9376/9376 [==============================] - 16s 2ms/step - loss: 0.6656 - accuracy: 0.5987
Epoch 2/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6477 - accuracy: 0.6309
Epoch 3/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6455 - accuracy: 0.6330
Epoch 4/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6402 - accuracy: 0.6364
Epoch 5/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6402 - accuracy: 0.6334
Epoch 6/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6388 - accuracy: 0.6398
Epoch 7/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6374 - accuracy: 0.6359
Epoch 8/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6368 - accuracy: 0.6387
Epoch 9/20
9376/9376 [==============================] - 12s 1ms/step - loss: 0.6353 - accuracy: 0.6368
Epoch 10/20
9376/9376 [==============================] - 12s 1ms/step - l

In [26]:
print(f"Macro avg f1 score without the MT dataset is {sum(f1_withoutMT)/len(f1_withoutMT)}")

Macro avg f1 score without the MT dataset is 0.5008008540438589


#### Leaving OC dataset and testing on it

In [27]:
f1_withoutOC=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=res_OC[res_OC["label"]==1]
    df_negative=res_OC[res_OC["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_OC.sentence.values
    y_test=df_OC.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_withoutOC.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_141"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_141 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_281 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_282 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_141 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_281 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_282 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________

Epoch 1/20
8196/8196 [==============================] - 15s 2ms/step - loss: 0.6754 - accuracy: 0.5691
Epoch 2/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6469 - accuracy: 0.6287
Epoch 3/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6454 - accuracy: 0.6312
Epoch 4/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6426 - accuracy: 0.6365
Epoch 5/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6405 - accuracy: 0.6370
Epoch 6/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6374 - accuracy: 0.6375
Epoch 7/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6352 - accuracy: 0.6436
Epoch 8/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6335 - accuracy: 0.6432
Epoch 9/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6313 - accuracy: 0.6409
Epoch 10/20
8196/8196 [==============================] - 11s 1ms/step - l

Model: "sequential_148"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_148 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_295 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_296 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_148 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_295 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_296 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
8196/8196 [==============================] - 16s 2ms/step - loss: 0.6709 - accuracy: 0.5828
Epoch 2/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6493 - accuracy: 0.6263
Epoch 3/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6454 - accuracy: 0.6301
Epoch 4/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6392 - accuracy: 0.6315
Epoch 5/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6432 - accuracy: 0.6323
Epoch 6/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6379 - accuracy: 0.6316
Epoch 7/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6388 - accuracy: 0.6315
Epoch 8/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6379 - accuracy: 0.6349
Epoch 9/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6340 - accuracy: 0.6391
Epoch 10/20
8196/8196 [==============================] - 11s 1ms/step - l

Model: "sequential_155"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_155 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_309 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_310 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_155 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_309 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_310 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
8196/8196 [==============================] - 16s 2ms/step - loss: 0.6788 - accuracy: 0.5559
Epoch 2/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6512 - accuracy: 0.6198
Epoch 3/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6470 - accuracy: 0.6281
Epoch 4/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6451 - accuracy: 0.6335
Epoch 5/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6430 - accuracy: 0.6369
Epoch 6/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6412 - accuracy: 0.6324
Epoch 7/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6382 - accuracy: 0.6397
Epoch 8/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6356 - accuracy: 0.6397
Epoch 9/20
8196/8196 [==============================] - 11s 1ms/step - loss: 0.6366 - accuracy: 0.6397
Epoch 10/20
8196/8196 [==============================] - 11s 1ms/step - l

In [28]:
print(f"Macro avg f1 score without the OC dataset is {sum(f1_withoutOC)/len(f1_withoutOC)}")

Macro avg f1 score without the OC dataset is 0.52318773263821


#### Leaving PE dataset and testing on it

In [29]:
f1_withoutPE=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=res_PE[res_PE["label"]==1]
    df_negative=res_PE[res_PE["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_PE.sentence.values
    y_test=df_PE.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_withoutPE.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_161"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_161 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_321 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_322 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_161 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_321 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_322 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________

Epoch 1/20
5414/5414 [==============================] - 13s 2ms/step - loss: 0.6940 - accuracy: 0.4972
Epoch 2/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6930 - accuracy: 0.5042
Epoch 3/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6826 - accuracy: 0.5602
Epoch 4/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6798 - accuracy: 0.5700
Epoch 5/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6782 - accuracy: 0.5737
Epoch 6/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6760 - accuracy: 0.5763
Epoch 7/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6748 - accuracy: 0.5864
Epoch 8/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6753 - accuracy: 0.5833
Epoch 9/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6726 - accuracy: 0.5820
Epoch 10/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.67

Model: "sequential_168"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_168 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_335 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_336 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_168 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_335 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_336 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
5414/5414 [==============================] - 13s 2ms/step - loss: 0.6939 - accuracy: 0.4996
Epoch 2/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6934 - accuracy: 0.5011
Epoch 3/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6897 - accuracy: 0.5375
Epoch 4/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6843 - accuracy: 0.5704
Epoch 5/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6826 - accuracy: 0.5720
Epoch 6/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6796 - accuracy: 0.5785
Epoch 7/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6781 - accuracy: 0.5864
Epoch 8/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6788 - accuracy: 0.5866
Epoch 9/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6765 - accuracy: 0.5870
Epoch 10/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.67

Model: "sequential_175"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_175 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_349 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_350 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_175 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_349 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_350 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
5414/5414 [==============================] - 14s 3ms/step - loss: 0.6938 - accuracy: 0.5022
Epoch 2/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6931 - accuracy: 0.5115
Epoch 3/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6943 - accuracy: 0.5002
Epoch 4/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6935 - accuracy: 0.4935
Epoch 5/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6930 - accuracy: 0.5081
Epoch 6/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6930 - accuracy: 0.5085
Epoch 7/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6922 - accuracy: 0.5235
Epoch 8/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6903 - accuracy: 0.5334
Epoch 9/20
5414/5414 [==============================] - 8s 1ms/step - loss: 0.6842 - accuracy: 0.5715
Epoch 10/20
5414/5414 [==============================] - 8s 2ms/step - loss: 0.68

In [30]:
print(f"Macro avg f1 score without the PE dataset is {sum(f1_withoutPE)/len(f1_withoutPE)}")

Macro avg f1 score without the PE dataset is 0.5433015765756154


#### Leaving the VG dataset and testing on it

In [31]:
f1_withoutVG=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=res_VG[res_VG["label"]==1]
    df_negative=res_VG[res_VG["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_VG.sentence.values
    y_test=df_VG.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_withoutVG.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_181"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_181 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_361 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_362 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_181 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_361 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_362 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________

Epoch 1/20
8506/8506 [==============================] - 18s 2ms/step - loss: 0.6614 - accuracy: 0.5952
Epoch 2/20
8506/8506 [==============================] - 12s 1ms/step - loss: 0.6371 - accuracy: 0.6381
Epoch 3/20
8506/8506 [==============================] - 12s 1ms/step - loss: 0.6345 - accuracy: 0.6392
Epoch 4/20
8506/8506 [==============================] - 12s 1ms/step - loss: 0.6366 - accuracy: 0.6472
Epoch 5/20
8506/8506 [==============================] - 12s 1ms/step - loss: 0.6364 - accuracy: 0.6440
Epoch 6/20
8506/8506 [==============================] - 12s 1ms/step - loss: 0.6334 - accuracy: 0.6465
Epoch 7/20
8506/8506 [==============================] - 12s 1ms/step - loss: 0.6309 - accuracy: 0.6493
Epoch 8/20
8506/8506 [==============================] - 12s 1ms/step - loss: 0.6310 - accuracy: 0.6460
Epoch 9/20
8506/8506 [==============================] - 12s 1ms/step - loss: 0.6271 - accuracy: 0.6508
Epoch 10/20
8506/8506 [==============================] - 12s 1ms/step - l

Model: "sequential_188"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_188 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_375 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_376 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_188 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_375 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_376 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
8506/8506 [==============================] - 20s 2ms/step - loss: 0.6939 - accuracy: 0.5043
Epoch 2/20
8506/8506 [==============================] - 14s 2ms/step - loss: 0.6924 - accuracy: 0.5139
Epoch 3/20
8506/8506 [==============================] - 14s 2ms/step - loss: 0.6607 - accuracy: 0.6058
Epoch 4/20
8506/8506 [==============================] - 14s 2ms/step - loss: 0.6468 - accuracy: 0.6280
Epoch 5/20
8506/8506 [==============================] - 14s 2ms/step - loss: 0.6419 - accuracy: 0.6400
Epoch 6/20
8506/8506 [==============================] - 14s 2ms/step - loss: 0.6432 - accuracy: 0.6249
Epoch 7/20
8506/8506 [==============================] - 14s 2ms/step - loss: 0.6362 - accuracy: 0.6398
Epoch 8/20
8506/8506 [==============================] - 14s 2ms/step - loss: 0.6337 - accuracy: 0.6406
Epoch 9/20
8506/8506 [==============================] - 14s 2ms/step - loss: 0.6298 - accuracy: 0.6411
Epoch 10/20
8506/8506 [==============================] - 14s 2ms/step - l

Model: "sequential_195"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_195 (Embedding)    (None, 20, 50)            182900    
_________________________________________________________________
lstm_389 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_390 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_195 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_389 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_390 (Dense)            (None, 1)                 9         
Total params: 205,637
Trainable params: 205,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
8506/8506 [==============================] - 19s 2ms/step - loss: 0.6618 - accuracy: 0.5978
Epoch 2/20
8506/8506 [==============================] - 13s 2ms/step - loss: 0.6399 - accuracy: 0.6440
Epoch 3/20
8506/8506 [==============================] - 13s 1ms/step - loss: 0.6342 - accuracy: 0.6465
Epoch 4/20
8506/8506 [==============================] - 13s 1ms/step - loss: 0.6315 - accuracy: 0.6465
Epoch 5/20
8506/8506 [==============================] - 13s 2ms/step - loss: 0.6328 - accuracy: 0.6433
Epoch 6/20
8506/8506 [==============================] - 13s 1ms/step - loss: 0.6331 - accuracy: 0.6454
Epoch 7/20
8506/8506 [==============================] - 13s 1ms/step - loss: 0.6320 - accuracy: 0.6454
Epoch 8/20
8506/8506 [==============================] - 13s 2ms/step - loss: 0.6263 - accuracy: 0.6507
Epoch 9/20
8506/8506 [==============================] - 13s 2ms/step - loss: 0.6277 - accuracy: 0.6499
Epoch 10/20
8506/8506 [==============================] - 13s 2ms/step - l

In [32]:
print(f"Macro avg f1 score without the VG dataset is {sum(f1_withoutVG)/len(f1_withoutVG)}")

Macro avg f1 score without the VG dataset is 0.5373235904456186


#### Leaving WD dataset and training on it

In [14]:
f1_withoutWD=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=res_WD[res_WD["label"]==1]
    df_negative=res_WD[res_WD["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_WD.sentence.values
    y_test=df_WD.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_withoutWD.append(result["macro avg"]["f1-score"])

Model 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            450650    
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 473,

Epoch 1/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6744 - accuracy: 0.5755
Epoch 2/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6521 - accuracy: 0.6312
Epoch 3/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6467 - accuracy: 0.6323
Epoch 4/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6451 - accuracy: 0.6333
Epoch 5/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6425 - accuracy: 0.6371
Epoch 6/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6382 - accuracy: 0.6398
Epoch 7/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6389 - accuracy: 0.6412
Epoch 8/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6358 - accuracy: 0.6402
Epoch 9/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6357 - accuracy: 0.6402
Epoch 10/20
9178/9178 [==============================] - 12s 1ms/step - l

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 50)            449200    
_________________________________________________________________
lstm_15 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_16 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_8 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 9         
Total params: 471,937
Trainable params: 471,937
Non-trainable params: 0
________________________________________________

Epoch 1/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6739 - accuracy: 0.5738
Epoch 2/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6519 - accuracy: 0.6213
Epoch 3/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6470 - accuracy: 0.6281
Epoch 4/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6437 - accuracy: 0.6265
Epoch 5/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6427 - accuracy: 0.6289
Epoch 6/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6410 - accuracy: 0.6335
Epoch 7/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6393 - accuracy: 0.6322
Epoch 8/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6381 - accuracy: 0.6357
Epoch 9/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6372 - accuracy: 0.6348
Epoch 10/20
9178/9178 [==============================] - 11s 1ms/step - l

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 20, 50)            456650    
_________________________________________________________________
lstm_29 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_30 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_15 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 9         
Total params: 479,387
Trainable params: 479,387
Non-trainable params: 0
_______________________________________________

Epoch 1/20
9178/9178 [==============================] - 12s 1ms/step - loss: 0.6759 - accuracy: 0.5699
Epoch 2/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6522 - accuracy: 0.6260
Epoch 3/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6463 - accuracy: 0.6288
Epoch 4/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6437 - accuracy: 0.6303
Epoch 5/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6418 - accuracy: 0.6333
Epoch 6/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6382 - accuracy: 0.6329
Epoch 7/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6366 - accuracy: 0.6365
Epoch 8/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6328 - accuracy: 0.6373
Epoch 9/20
9178/9178 [==============================] - 11s 1ms/step - loss: 0.6309 - accuracy: 0.6432
Epoch 10/20
9178/9178 [==============================] - 11s 1ms/step - l

In [15]:
print(f"Macro avg f1 score without the WD dataset is {sum(f1_withoutWD)/len(f1_withoutWD)}")

Macro avg f1 score without the WD dataset is 0.4086175486034345


#### Leaving the WTP dataset and testing on it

In [17]:
f1_withoutWTP=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=res_WTP[res_WTP["label"]==1]
    df_negative=res_WTP[res_WTP["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_WTP.sentence.values
    y_test=df_WTP.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_withoutWTP.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 20, 50)            376700    
_________________________________________________________________
lstm_41 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_42 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_21 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 9         
Total params: 399,437
Trainable params: 399,437
Non-trainable params: 0
_______________________________________

Epoch 1/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6670 - accuracy: 0.5873
Epoch 2/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6398 - accuracy: 0.6420
Epoch 3/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6335 - accuracy: 0.6471
Epoch 4/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6281 - accuracy: 0.6473
Epoch 5/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6282 - accuracy: 0.6445
Epoch 6/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6261 - accuracy: 0.6490
Epoch 7/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6250 - accuracy: 0.6562
Epoch 8/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6218 - accuracy: 0.6533
Epoch 9/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6189 - accuracy: 0.6528
Epoch 10/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.619

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 20, 50)            380500    
_________________________________________________________________
lstm_55 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_56 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_28 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_56 (Dense)             (None, 1)                 9         
Total params: 403,237
Trainable params: 403,237
Non-trainable params: 0
_______________________________________________

Epoch 1/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6752 - accuracy: 0.5559
Epoch 2/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6360 - accuracy: 0.6417
Epoch 3/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6321 - accuracy: 0.6492
Epoch 4/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6291 - accuracy: 0.6540
Epoch 5/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6283 - accuracy: 0.6491
Epoch 6/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6256 - accuracy: 0.6554
Epoch 7/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6259 - accuracy: 0.6524
Epoch 8/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6207 - accuracy: 0.6565
Epoch 9/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.6183 - accuracy: 0.6598
Epoch 10/20
7330/7330 [==============================] - 8s 1ms/step - loss: 0.619

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, 20, 50)            375750    
_________________________________________________________________
lstm_69 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_70 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_35 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_70 (Dense)             (None, 1)                 9         
Total params: 398,487
Trainable params: 398,487
Non-trainable params: 0
_______________________________________________

Epoch 1/20
7330/7330 [==============================] - 10s 1ms/step - loss: 0.6647 - accuracy: 0.5900
Epoch 2/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6364 - accuracy: 0.6490
Epoch 3/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6328 - accuracy: 0.6523
Epoch 4/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6305 - accuracy: 0.6531
Epoch 5/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6270 - accuracy: 0.6568
Epoch 6/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6286 - accuracy: 0.6610
Epoch 7/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6258 - accuracy: 0.6638
Epoch 8/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6200 - accuracy: 0.6573
Epoch 9/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.6231 - accuracy: 0.6596
Epoch 10/20
7330/7330 [==============================] - 9s 1ms/step - loss: 0.61

In [18]:
print(f"Macro avg f1 score without the WTP dataset is {sum(f1_withoutWTP)/len(f1_withoutWTP)}")

Macro avg f1 score without the WTP dataset is 0.5113097971614836
